<a href="https://colab.research.google.com/github/ameyas1/QA_Summarization/blob/master/Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install beautifulsoup4
!pip install google

!pip install html5lib

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
from transformers import *
from IPython.display import clear_output
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80)

try: 
    from googlesearch import search 
except ImportError:  
    print("No module named 'google' found") 

import requests
from bs4 import BeautifulSoup

import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

import string



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from transformers import TFAutoModelWithLMHead, AutoTokenizer

model = TFAutoModelWithLMHead.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# ARTICLE = "I can confidently say that God's Gift - 14 Days is going to be ONE OF THE BEST K-dramas of 2014!! I can already see many awards coming its way at the end of the year!!\
# The story line is just amazing. From episode 1, I was captivated by the plot, making me sit at the edge of my chair, anticipating of what would happen next. Just when I thought I figured out who is behind it all, something unexpected happened and I was left at a dead end with numerous questions. Even until the very last minute of the final episode I was left awestruck. It took me a few minutes to register what happened and then everything clicked into place. Usually I am pretty good at figuring out who is bad and who is good, but this drama totally made me run around in circles.\
# The cast was also amazing. I have no words to describe how well everyone played their parts. Lee Bo Young played the mother soo well. The determination she showed to save her child no matter what, it felt real. She is clearly a great actress. And what can I say about Cho Seung Woo, he was just amazing!! The action scenes he did was just top notch!! Seriously can i have him protect me too? haha But in all seriousness, he did a great job!"

# # T5 uses a max_length of 512 so we cut the article to 512 tokens.
# inputs = tokenizer.encode("summarize: " + ARTICLE, return_tensors="tf", max_length=512)
# outputs = model.generate(inputs, max_length=200, min_length=80, length_penalty=2.0, num_beams=4, early_stopping=True)
# print(outputs)

All model checkpoint weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
inputs = tokenizer.encode("summarize: " + ARTICLE, return_tensors="tf", max_length=512)
outputs = model.generate(inputs, max_length=200, min_length=80, length_penalty=1.0, num_beams=4, early_stopping=True)

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(outputs[0]))

"God's Gift - 14 days is going to be ONE of the BEST K-dramas of 2014 . the story line is just amazing . the cast was also amazing . the action scenes he did was just top notch!! Seriously can i have him protect me too? haha But in all seriousness, he did a great job! ."

In [ ]:
def answer(ques):
  ans =''
  all_ans = []
  max_score = -np.inf
  query = ques
  urls = [] 
  for j in search(query, tld="com", num=3, stop=3, pause=2):
    urls.append(j)

  keywords = query.split(" ")

  stop_words = set(stopwords.words('english')) 
  keys = [w for w in keywords if not w in stop_words] 

  words = [i for i in keys if i not in string.punctuation]

  contexts=[]

  for URL in urls:
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html5lib')

    for p in soup.select('p'):
      context = p.getText()
      for keyword in words:
        if keyword.lower() in context.lower() and context not in contexts:
          contexts.append(context)
          inputs = tokenizer.encode("summarize: " + context, return_tensors="tf", max_length=512, truncation=True)
          outputs = model.generate(inputs, max_length=200, min_length=80, length_penalty=1.0, num_beams=4, early_stopping=True)
          ans = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(outputs[0]))
          if '[UNK]' in ans or '[CLS]' in ans or '[SEP]' in ans:
            continue
          if ans in all_ans:
            continue
          all_ans.append(wrapper.fill(ans))
          print(ans)

  clear_output()

  print('All answers')
  all_ans =list(set(all_ans))
  for i, ANS in enumerate(all_ans):
    print(i," ",wrapper.fill(ANS))

In [ ]:
answer("What are the reviews of Signal Korean Drama ?")

All answers
0   "Signal" rides on something more elusive and invisible . the show rides on the
waves of superb writing and emotionally deep characters . "signal" didn't even
need "kisses or cringy lines to sweep hearts" as one fan put it . "signal"
didn't even need "kisses or cringy lines to sweep hearts"
1   tvN’s “Signal” ran for sixteen episodes from January 22 to March 12, 2016 . it
was written by veteran mystery writer Kim Eun Hee and directed by Kim Won Suk .
it became one of the highest rated dramas in the history of Korean cable
television history . there was a Japanese remake of the show which aired this
year .
2   audience members could easily follow the timeline’s and the details stuffed into
it without having our brains hurt . everything still being very grounded and
established . jae Han’s disappearance and death tied into the story arc of
corrupt higher ups and people of influence . audience members could follow the
timeline’s and the details stuffed into it without havin